In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from collections import Counter
from IPython.core.interactiveshell import InteractiveShell
import warnings
from pandas import read_csv
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn import preprocessing
from statsmodels.genmod.generalized_linear_model import GLM
from pandas.plotting import scatter_matrix
from scipy.stats import boxcox

InteractiveShell.ast_node_interactivity = "all"
#pd.set_option('precision', 3)
display.precision = 3
sns.set()
warnings.filterwarnings('ignore')

In [17]:
df_results = pd.read_csv("Datathon_Results_MOBILITY_2022_original_Students.csv", delimiter=",")


In [19]:
df_results.shape

(1448, 22)

In [21]:
def data_explore(dataframe):
    print("DATA EXPLORATION")
    print('*'*70)
    print("Shape of dataset : ",dataframe.shape)
    print('*'*70)
    print(dataframe.info())
    print('*'*70)
    print("STATISTICAL ANALYSIS OF NUMERICAL DATA")
    print('*'*70)
    print(dataframe.describe().T)
    print('*'*70)
    print("STATISTICAL ANALYSIS OF CATEGORICAL DATA")
    print('*'*70)
    print(dataframe.describe(exclude = ['float', 'int64']).T)
    print('*'*70)
    print("MISSING VALUES")
    print('*'*70)
    print(dataframe.isna().sum().sort_values(ascending=False))
    print('*'*70)
    print("MISSING VALUES IN %")
    print('*'*70)
    print(round(100* (dataframe.isnull().sum() / len(dataframe)).sort_values(ascending=False),2))
    print('*'*70)

In [22]:
df_results.columns = ["id","id_2","gender","center","year","days","postal","s1","s2","s3","same_route",
                      "s1back","s2back","s3back","r_fastest","r_cheapest","r_confortable","r_onlyoption",
                      "r_environmental","r_healthiest","r_needprivate","r_other"]

df_results = df_results.iloc[:, 2:]


In [23]:
df_results = df_results.dropna(subset=["postal"])
df_results["gender"].unique()

array(['Man', 'Woman', 'Non binary', 'NR/DK'], dtype=object)

In [24]:
df_results.shape

(1400, 20)

In [25]:

df_results['center'] = df_results['center'].str.extract(r'\((.*?)\)')
df_results["center"]

0        ESEIAAT
1          ETSAB
2         ETSEIB
3          EPSEM
4          ETSAB
          ...   
1443         FIB
1444       EPSEB
1445         FME
1446        FOOT
1447    ETSECCPB
Name: center, Length: 1400, dtype: object

In [26]:
df_results["same_route"].unique()

array(['No', 'Yes'], dtype=object)

In [27]:
trajectes = ["s1","s2","s3","s1back","s2back","s3back"]

for item in trajectes:
    df_results.loc[df_results[item] == 'Combustion vehicle (non-plug-in hybrid, electric or plug-in hybrid with non-renewable source charging),', item] = 'private'
    df_results.loc[df_results[item] == 'Renfe', item] = 'public'
    df_results.loc[df_results[item] == 'On foot', item] = 'active'
    df_results.loc[df_results[item] == 'Underground', item] = 'public'
    df_results.loc[df_results[item] == 'FGC', item] = 'public'
    df_results.loc[df_results[item] == 'Bicycle', item] = 'active'
    df_results.loc[df_results[item] == 'Tram', item] = 'public'
    df_results.loc[df_results[item] ==  'Scooter (or other micro-mobility devices) with renewable charging', item] = 'private'
    df_results.loc[df_results[item] == 'Taxi',item] = 'private'
    df_results.loc[df_results[item] == 'Electric vehicle (with Zero label and renewable source charging)', item] = 'private'
    df_results.loc[df_results[item] == 'Scooter (or other micro-mobility devices) with non-renewable charging', item] = 'private'
    df_results.loc[df_results[item] == 'Electric motorcycle', item] = 'private'
    df_results.loc[df_results[item] == 'Combustion or electric motorcycle with non-renewable source charging', item] = 'private'
    df_results.loc[df_results[item] == 'Bus', item] = 'public'



In [28]:
df_results["same_route"].unique()

array(['No', 'Yes'], dtype=object)

In [29]:
relevant_columns = ['s1', 's2', 's3', 's1back', 's2back', 's3back']

df_results[relevant_columns] = df_results[relevant_columns].replace('nan', np.nan)

In [30]:
def sum_non_nan(row):
    non_null_count = sum(1 if pd.notnull(row[col]) else 0 for col in relevant_columns)
    return non_null_count

# Apply the function row-wise to create a new column 'sum_non_nan_values'
df_results['total_travels'] = df_results.apply(sum_non_nan, axis=1)

In [31]:
df_results['total_travels'].head()

0    4
1    3
2    3
3    3
4    2
Name: total_travels, dtype: int64

In [32]:
# For each S1, ... S6, add the total values of value counts together

private_counts = df_results[relevant_columns].apply(lambda row: row.eq('private').sum(), axis=1)
public_counts = df_results[relevant_columns].apply(lambda row: row.eq('public').sum(), axis=1)
active_counts = df_results[relevant_columns].apply(lambda row: row.eq('active').sum(), axis=1)

df_results['private_ratio'] = private_counts / df_results['total_travels']
df_results['public_ratio'] = public_counts / df_results['total_travels']
df_results['active_ratio'] = active_counts / df_results['total_travels']

comprovar = df_results[["private_ratio",'public_ratio','active_ratio',"total_travels"]]
comprovar.head()

,private_ratio,public_ratio,active_ratio,total_travels
0,0.250000,0.500000,0.25,4
1,1.000000,0.000000,0.00,3
2,0.333333,0.666667,0.00,3
3,0.000000,0.000000,1.00,3
4,0.000000,1.000000,0.00,2


In [33]:
df_results.shape

(1400, 24)

In [34]:
df_results = df_results.dropna(subset=["postal"])
df_results["gender"].unique()

array(['Man', 'Woman', 'Non binary', 'NR/DK'], dtype=object)

In [35]:
trajectes = ["s1","s2","s3","s1back","s2back","s3back"]

for item in trajectes:
    df_results.loc[df_results[item] == 'Combustion vehicle (non-plug-in hybrid, electric or plug-in hybrid with non-renewable source charging),', item] = 'private'
    df_results.loc[df_results[item] == 'Renfe', item] = 'public'
    df_results.loc[df_results[item] == 'On foot', item] = 'active'
    df_results.loc[df_results[item] == 'Underground', item] = 'public'
    df_results.loc[df_results[item] == 'FGC', item] = 'public'
    df_results.loc[df_results[item] == 'Bicycle', item] = 'active'
    df_results.loc[df_results[item] == 'Tram', item] = 'public'
    df_results.loc[df_results[item] ==  'Scooter (or other micro-mobility devices) with renewable charging', item] = 'private'
    df_results.loc[df_results[item] == 'Taxi'] = 'private'
    df_results.loc[df_results[item] == 'Electric vehicle (with Zero label and renewable source charging)', item] = 'private'
    df_results.loc[df_results[item] == 'Scooter (or other micro-mobility devices) with non-renewable charging', item] = 'private'
    df_results.loc[df_results[item] == 'Electric motorcycle', item] = 'private'
    df_results.loc[df_results[item] == 'Combustion or electric motorcycle with non-renewable source charging', item] = 'private'
    df_results.loc[df_results[item] == 'Bus', item] = 'public'



In [36]:
def sum_non_nan(row):
    non_null_count = sum(1 if pd.notnull(row[col]) else 0 for col in relevant_columns)
    return non_null_count

# Apply the function row-wise to create a new column 'sum_non_nan_values'
df_results['total_travels'] = df_results.apply(sum_non_nan, axis=1)

In [37]:
# For each S1, ... S6, add the total values of value counts together

private_counts = df_results[relevant_columns].apply(lambda row: row.eq('private').sum(), axis=1)
public_counts = df_results[relevant_columns].apply(lambda row: row.eq('public').sum(), axis=1)
active_counts = df_results[relevant_columns].apply(lambda row: row.eq('active').sum(), axis=1)

df_results['private_ratio'] = private_counts / df_results['total_travels']
df_results['public_ratio'] = public_counts / df_results['total_travels']
df_results['active_ratio'] = active_counts / df_results['total_travels']


In [38]:
data_explore(df_results)

DATA EXPLORATION
**********************************************************************
Shape of dataset :  (1400, 24)
**********************************************************************
<class 'pandas.core.frame.DataFrame'>
Index: 1400 entries, 0 to 1447
Data columns (total 24 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   gender           1400 non-null   object 
 1   center           1400 non-null   object 
 2   year             1400 non-null   object 
 3   days             1400 non-null   int64  
 4   postal           1400 non-null   float64
 5   s1               1332 non-null   object 
 6   s2               1082 non-null   object 
 7   s3               889 non-null    object 
 8   same_route       1400 non-null   object 
 9   s1back           156 non-null    object 
 10  s2back           135 non-null    object 
 11  s3back           106 non-null    object 
 12  r_fastest        1400 non-null   object 
 13  r_cheapest   

In [39]:
df_results = df_results.dropna(subset=["private_ratio"])

In [40]:
data_explore(df_results)

DATA EXPLORATION
**********************************************************************
Shape of dataset :  (1381, 24)
**********************************************************************
<class 'pandas.core.frame.DataFrame'>
Index: 1381 entries, 0 to 1447
Data columns (total 24 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   gender           1381 non-null   object 
 1   center           1381 non-null   object 
 2   year             1381 non-null   object 
 3   days             1381 non-null   int64  
 4   postal           1381 non-null   float64
 5   s1               1332 non-null   object 
 6   s2               1082 non-null   object 
 7   s3               889 non-null    object 
 8   same_route       1381 non-null   object 
 9   s1back           156 non-null    object 
 10  s2back           135 non-null    object 
 11  s3back           106 non-null    object 
 12  r_fastest        1381 non-null   object 
 13  r_cheapest   

In [41]:
df_results = df_results.drop(columns = ["s1",'s2','s3','s1back','s2back','s3back','r_other'])

In [43]:
df_results.to_csv("preprocessed_be.csv", index=False)

TypeError: 'Index' object is not callable

In [ ]:
data_explore(df_results)


In [ ]:
data_explore(df_students2)